In [1]:
from web_scraper.web_scraper import CompendiumScraper
import pandas as pd
from selenium.webdriver.common.by import By
import psycopg2
import subprocess
import yaml
import sqlalchemy as sa
import time
import re

from selenium.webdriver.common.action_chains import ActionChains

In [2]:
with open(r'\Users\jasre\Code\ws_config.yml', 'r') as yml:
    config = yaml.safe_load(yml)

# Running To Do
- Parse follow ups for JP versions of characters (manual...)
- Set up DBT
- Set up marts
- Set up Streamlit

In [3]:
cs = CompendiumScraper('\\Users\\jasre\\Code\\ws_config.yml')

2023-09-04 13:44:41,261 - selenium.webdriver.common.selenium_manager - Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)
2023-09-04 13:44:43,726 - web_scraper.web_scraper - ====================================================
2023-09-04 13:44:43,726 - web_scraper.web_scraper - Retrieved all main character links. Generating remaining links for each character.
2023-09-04 13:44:43,726 - web_scraper.web_scraper - ====================================================
2023-09-04 13:44:43,826 - web_scraper.web_scraper - self.character_dict_omnibus entry for JIHLNABAAT was successful
2023-09-04 13:44:43,879 - web_scraper.web_scraper - self.character_dict_omnibus entry for WEISS was successful
2023-09-04 13:44:43,899 - web_scraper.web_scraper - self.character_dict_omnibus entry for ASTOS was successful
2023-09-04 13:44:43,970 - web_scraper.web_scraper - self.character_dict_omnibus entry for IRIS was successful
2023-09-04 13:44:43,990 - web_scraper.web_scrape

In [4]:
char_name = 'amarant'

ability_df = cs.generate_ability_df(char_name=char_name, JP=False)

ability_df

2023-09-04 13:44:47,208 - web_scraper.web_scraper - ====================================================
2023-09-04 13:44:47,209 - web_scraper.web_scraper - Parsing ability dict to df for AMARANT.
2023-09-04 13:44:47,209 - web_scraper.web_scraper - ====================================================
2023-09-04 13:44:47,210 - web_scraper.web_scraper - Couldn't find ability_dict for AMARANT. Creating it now.
2023-09-04 13:44:47,211 - web_scraper.web_scraper - ====================================================
2023-09-04 13:44:47,211 - web_scraper.web_scraper - Generating ability dictionary for AMARANT
2023-09-04 13:44:47,212 - web_scraper.web_scraper - ====================================================
2023-09-04 13:44:52,451 - web_scraper.web_scraper - Unable to access abilities for AMARANT.
2023-09-04 13:44:52,451 - web_scraper.web_scraper - They might not be released to GL yet.
2023-09-04 13:44:52,451 - web_scraper.web_scraper - Couldn't generate ability df for GL version of the 

In [4]:
bt_df = cs.retrieve_hp_caps_from_bt(char_name=char_name, JP=True, return_output=True)

bt_df

2023-09-04 11:27:43,382 - web_scraper.web_scraper - ====================================================
2023-09-04 11:27:43,383 - web_scraper.web_scraper - Retrieving BT info for RUFUS.
2023-09-04 11:27:43,383 - web_scraper.web_scraper - ====================================================
2023-09-04 11:27:56,365 - web_scraper.web_scraper - Standard BT stack slider div-class not found. Trying second possibility.
2023-09-04 11:27:56,370 - web_scraper.web_scraper - No stack slider found. Assuming rufus has a BT without stacks.
2023-09-04 11:27:56,432 - web_scraper.web_scraper - Retrieved BT info for RUFUS.


,char_name,bt_personal_hp_dmg_cap_up,bt_party_hp_dmg_cap_up,enemy_count_apply_list,game_version
0,rufus,0,30,"[1, 2, 3]",JP


In [6]:
ha_df = cs.retrieve_ha_hp_dmg_cap_up(char_name=char_name, JP=True, return_output=True)

ha_df

2023-09-04 11:28:55,132 - web_scraper.web_scraper - ====================================================
2023-09-04 11:28:55,133 - web_scraper.web_scraper - Retrieving High Armor info for RUFUS.
2023-09-04 11:28:55,133 - web_scraper.web_scraper - ====================================================
2023-09-04 11:29:05,985 - web_scraper.web_scraper - High armor info parsed for RUFUS.


,char_name,personal_hp_dmg_cap_up,party_ha_hp_dmg_cap_up,game_version
0,rufus,45,0,JP


# Following can be used to manually upload data to Postgres if the web scraper fails to do so. 
## DO NOT DELETE!

In [ ]:
# SQL alchemy stuff

engine_url = sa.URL.create(
        "postgresql",
        username=config['pg_user'],
        password=config['pg_pass'],
        host=config['pg_host'],
        database=config['pg_db']
    )
    
engine = sa.create_engine(engine_url)

bt_effects = pd.read_csv(config['datasets_dir'] + 'raw_bt_effects.csv')
abilities = pd.read_csv(config['datasets_dir'] + 'raw_abilities.csv')
ha_caps = pd.read_csv(config['datasets_dir'] + 'raw_high_armor_caps.csv')

with engine.begin() as conn:
    abilities.to_sql('raw_abilities', con=conn, index=False, if_exists='append')
    bt_effects.to_sql('raw_bt_effects', con=conn, index=False, if_exists='append')
    ha_caps.to_sql('raw_high_armor_caps', con=conn, index=False, if_exists='append')